## Data Preprrocessing
Author: Dalyah Aljamal, April 21, 2018
#This file read data (Training (X,Y) , testing(X,Y))
#Perform MiniBatching
#Image preprocessing: 
1. Segmentation (not complete)
2. Resizing
3. Normalization (not complete)


#Dp preprossing for all data once and Save all the data in a folder (ready_data) (notcomplete)

In [47]:
using Plots
using Knet;
using Compat,GZip 
using Images
using Colors
using Images
using ImageSegmentation


In [1]:
function seg_image(img; training=true)
    
    xDim= size(img,1);
    yDim= size(img,2);
    min_size= 100;
    segments = felzenszwalb(img, 300, min_size); # k=300 (the merging threshold), min_size = 100 (smallest number of pixels/region)

    seg_img=Any[]
    ymax=0;
    ymin=10000;
    xmax=0;
    xmin=10000;
    
    #num of different img segements
    num_labels= maximum(segments.image_indexmap);
    
    while (num_labels==1 && min_size> 40) #There is a need to lower felzenszwalb min_size parameter
        min_size=min_size-10;
        segments = felzenszwalb(img, 300, min_size-10);
        num_labels= maximum(segments.image_indexmap);
    end
        
    #Getting hand label index "hand_i"
    if (training && num_labels>1 )
        hand_i= 2; 
    else
        println("calculating segm label")
        #println(segments.image_indexmap)
        #println(maximum(segments.image_indexmap))
        if (num_labels==1)
            hand_i=1
        else
        num= indmax([segment_pixel_count(segments,i) for i in 2:num_labels])
        println("Passed indmax");
        println(num);
        hand_i= num+1;
        end
    end
    println(hand_i)
    
    #Getting max and min of hand segement
    for i in 1:xDim
        seg_cols=Any[]
        for j in 1:yDim
        
            if labels_map(segments)[i,j] == hand_i
                if i > xmax
                    xmax=i;
                end
                if i < xmin
                    xmin=i;
                end
                if j > ymax
                    ymax=j;
                end
                if j < ymin
                    ymin=j;
                end
                
            end
        end
            
    end
    

    #function get_random_color(seed)
      #  srand(seed)
      #  rand(RGB{N0f8})
    #end
    #3map(i->get_random_color(i), labels_map(segments))
    seg_img= img[xmin:xmax, ymin:ymax]
    return seg_img
   
end
   

seg_image (generic function with 1 method)

In [20]:
#Reading Training and Testing img labels

#HOW THE DATA IS ORGANIZED

#Each line is corresponding to one image.
#Each line has 16x3 numbers, which indicates (x, y, z) of 16 joint locations. Note that these are joint CENTRE locations.
#Note that (x, y) are in pixels and z is in mm.
#The order of 16 joints is Palm, Thumb root, Thumb mid, Thumb tip,
#Index root, Index mid, Index tip, Middle root, Middle mid, Middle tip, Ring root,
#Ring mid, Ring tip, Pinky root, Pinky mid, Pinky tip.

function read_labels(label_path)
    labels=Any[]
    open(label_path) do f
         # loop through lines in file (images)
            #loop through all words in a line (joint locations)
                #skip first word
                #stack all joint locations on top of each other
                #return imgjoints vector(48,1) or (49, 1) as first element is the name of img
            #horizantally concatanate imgjoints into labels matrix (48, #lines)
        for ln in eachline(f)
            if(ln=="")
                continue;
            end
            img_joints= Array{Float32}(48)
            word= split(ln)
            for i in 2:49
                w=parse(Float32, word[i])
                img_joints[i-1] = w
            end
            if length(labels) ==0
                labels=img_joints
            else
                labels= hcat(labels, img_joints)
            end
        end
        
    end
    return labels
end



read_labels (generic function with 1 method)

In [ ]:
#Extracting Data Zip Folders
using InfoZIP
using ZipFile
#Since Training file was corupted
#$ zip -FFv Training.zip --out fixed.zip
#InfoZIP.unzip("fixed.zip", "Data/")
#InfoZIP.unzip("Training.zip", "Data/")
#InfoZIP.unzip("Testing.zip", "Data/")

#They are already extracted on my AWS image: HandPosE_Dalyah_v6

In [2]:
## read images function
function read_img(rootpath, arr1,arr2, Atype)
images = Any[]

for i =1:length(arr2)
    for j=1:length(arr1)
        path2 = string(rootpath,"/",arr1[j],"/",arr2[i])
        x =readdir(path2) #save all img names in arr1[i]/arr2[j] in x var
        for k=1:length(x)
            println("trn image :",arr1[j],"/", arr2[i],"/",x[k]) 
            path3 = string(path2,"/",x[k])
            img = load(path3)
            #Transfer to gray scale
            img2=seg_image(img)
    
            #Resize img to (128,128)
            sz=(128,128)
            σ = map((o,n)->0.75*o/n, size(img2), sz)
            kern = KernelFactors.gaussian(σ)   # from ImageFiltering
            img3 = imresize(imfilter(img2, kern, NA()), sz)
            
            #Correct Dimentionality
            img4=reshape(img3,(128,128,1,1))
            img5= convert(Array{Float32},img4)
            #Normalization from -1 to 1
            img6= 2*(img5 - minimum(img5)) / (maximum(img5) - minimum(img5) )- 1
        
            println(summary(img6))
            #img5= convert(Atype,img5)
            push!(images,img6)
        end
    end
end
    images=cat(4,images...)
    #images2=convert(Atype, images)
    return images
end


read_img (generic function with 1 method)

In [3]:
function read_tst_img(rootpath, arr1, Atype)
images = Any[]
for i =1:length(arr1)
        path2 = string(rootpath,"/",arr1[i])
        x =readdir(path2) #save all img names in arr1[i]/arr2[j] in x var
        for k=1:length(x)
            println("tst image :", arr1[i],"/",x[k]) 
            path3 = string(path2,"/",x[k])
            img = load(path3)
            
            #Segment the image
            img2=seg_image(img,training=false)
            
            #Resize img to (128,128)
            sz=(128,128)
            σ = map((o,n)->0.75*o/n, size(img2), sz)
            kern = KernelFactors.gaussian(σ)   # from ImageFiltering
            img3 = imresize(imfilter(img2, kern, NA()), sz)
            
            #correct Dimentionality
            img4=reshape(img3,(128,128,1,1))
            img5= convert(Array{Float32},img4)
            #Normalization from -1 to 1
            img6= 2*(img5 - minimum(img5)) / (maximum(img5) - minimum(img5) )- 1
            println(summary(img6))
            #img5= convert(Atype,img5)
            push!(images,img6)
        
    end
end
    images=cat(4,images...)
    #images2=convert(Atype, images)
    return images
end


read_tst_img (generic function with 1 method)

In [44]:
#Mini Batching


function dataset_minibatching(BS, data, labels, Atype)
    all_minibatches=Any[]
    for i in 1:BS:size(data,4)
        b1 = min(i+BS-1, size(data, 4))
        batch=data[:,:,:,i:b1]
        batch=convert(Atype, batch)
        lab= convert(Atype, labels[:,i:b1])
        push!(all_minibatches, (batch, lab))
    end
    
    return  all_minibatches #u can acess single batch by all_minibatches[i]
end



dataset_minibatching (generic function with 1 method)

## Preparing all Data

In [ ]:
#trn_labelpath= "Data/Training/labels.txt"
#tst1_labelpath= "Data/Testing/test_seq_1.txt"
#tst2_labelpath= "Data/Testing/test_seq_2.txt"

#trn_labels= read_labels(trn_labelpath) # (48, )
#tst1_labels= read_labels(tst1_labelpath) # (48, )
#tst2_labels= read_labels(tst2_labelpath) # (48, )

#println(map(summary,(trn_labels, tst1_labels, tst2_labels)))

In [ ]:
#rootpath="imgDraft" # should change based on path on Machine you work on
#arr1 = ["1","2"]
#arr2 = ["123","321"]
#images= read_img(rootpath,arr1,arr2);
#for x in images
  #  println(size(x))
  #  println(typeof(x))
#end
#println(length(images))

In [ ]:
#Minibatch all dataset & save it in ready_data folder
#link for saving
# https://github.com/JuliaIO/JLD.jl